In [1]:
# 연습문제 : 기상청 날씨 정보 수집하기
# http://www.weather.go.kr/weather/climate/past_cal.jsp?stn=108&yy=2018&mm=4&obs=9&x=23&y=5

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  # pip install numpy 를 해야 사용가능함
import pandas as pd  # pip install pandas 해야 사용가능하고 numpy 가 먼저 설치되어 있어야 함
import xlwt

import random
import os   # 폴더 생성하기 위해 필요함

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("기상청 날씨 정보 수집하기")
print("=" *80)

query_txt = '기상청날씨정보'

start_date = input('1.조회를 시작할 날짜를 입력하세요(예:20180101): ')
end_date = input('2.조회를 종료할 날짜를 입력하세요(예:20181231): ')
f_dir = input("3.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\data\\):")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
#fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )


기상청 날씨 정보 수집하기
1.조회를 시작할 날짜를 입력하세요(예:20180101): 20180801
2.조회를 종료할 날짜를 입력하세요(예:20181231): 20190730
3.결과 파일을 저장할 폴더명만 쓰세요(예:c:\data\):c:\data\


In [2]:
#Step 4. 날짜 계산하기

s_year=start_date[0:4]
e_year= end_date[0:4]

s_mon=start_date[4:6]
e_mon=end_date[4:6]

s_day=start_date[6:8]
e_day=end_date[6:8]

print('s_year:',s_year)
print('e_year:',e_year)
print('s_mon:', s_mon)
print('e_mon:',e_mon)
print(s_day)
print(e_day)

mon=["01","02","03","04","05","06","07","08","09","10","11","12"]
i=0

start_date2=[]
end_date2=[]


#while int(s_year) <= int(e_year) :
if s_year < e_year :
    for a in range(0,len(mon)) :
        if mon[a] >= s_mon:
            s_date = str(s_year)+mon[a]
            start_date2.append(s_date)
    s_year2 = int(s_year) + 1
        
if  int(s_year2) > int(s_year) and int(s_year2) < int(e_year) :
    for b in range(0,len(mon)) :
            s_date2 = str(s_year2) + mon[b]
            start_date2.append(s_date2)

if int(s_year2) == int(e_year):
    for c in range(0,len(mon)) :
        if mon[c] <= e_mon :
            s_date3 = str(s_year2) + mon[c]
            start_date2.append(s_date3)
              
#print(start_date2)

#Step 5. 각 날짜별 날씨 정보를 추출합니다.
path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)


year2 = []  # 년을 저장하는 리스트
mon2 = []   # 월 을 저장하는 리스트   
day1 = []   # 요일을 저장하는 리스트
day2 = []   # 일을 저장하는 리스트
weather2 =[] # 날씨를 저장하는 리스트

url_1 = 'http://www.weather.go.kr/weather/climate/past_cal.jsp?stn=108&yy='
url_2 = '&mm='
url_3='&obs=9&x=23&y=5'

for x in range(0,len(start_date2)) :
        s_year2 = start_date2[x][0:4] 
        s_mon2 = start_date2[x][4:]
        full_url = url_1+s_year2+url_2+s_mon2+url_3
       
        print("\n")
        print("%s년 %s월의 날씨 수집 중 ===================================" %(s_year2,s_mon2))
        print(full_url)
        
        driver.get(full_url)
        time.sleep(5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    
        count = 0
        
        # 요일 추출하기 : day1 변수 사용
        #day1=[]
        d2 = []
        d1= soup.select('table.table_develop > thead > tr')
        #print(d1[0].get_text())
        if s_mon2 =="02" :
            for d in range(0,4) :
                d2.append(d1[0].get_text())    
        elif s_mon2 == "04" or s_mon2=="06" or s_mon2=="09" or s_mon2=="11" :
            for d in range(0,5) :
                d2.append(d1[0].get_text()) 
        else :
            for d in range(0,6) :
                d2.append(d1[0].get_text()) 
            
        #print(d2)
        d3=[]
        for i in range(0,len(d2)):
            d3.append(d2[i].split("\n"))
        #print(d3)
        d4 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, d3) for i in b]
        #print(d4)
        
        for d in range(0,len(d4)) :
            if len(d4[d]) > 1:
                day1.append(d4[d])
        #print(day1)

       # 일 추출하기    = day2 변수에 최종 요일 저장
        w2 =[]
        w5 =[]
        w1= soup.select('table.table_develop > tbody > tr')
        
        for i in range(0,11) :
            if i % 2 == 0 :
                #print(w1[i].get_text().replace("\n",""))
                try :
                    w2.append(w1[i].get_text())
                except :
                    continue
        #print(w2)
        w3=[]
        for i in range(0,len(w2)):
            w3.append(w2[i].split("\n"))
        #print(day_4)
    
        w4 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, w3) for i in b]
        #print(day5)
        
        # \xa0  기호의 갯수 찾기
        blank_cnt=0
        for d in range(0,len(w4)) :
            if len(w4[d]) == 1:
                blank_cnt += 1
        #print('blank 갯수:',blank_cnt)
        
        for d in range(0,len(w4)) :
            if len(w4[d]) > 0:
                day2.append(w4[d])
                #print(w4[d])
        

        # 년 / 월 추가하기:

        if s_mon2 =="02" :
            p_date = 28 + int(blank_cnt)
            for y in range(0,p_date) :
                year2.append(s_year2)
                mon2.append(s_mon2)    
        elif s_mon2 == "04" or s_mon2=="06" or s_mon2=="09" or s_mon2=="11" :
            p_date = 30 + int(blank_cnt)
            for y in range(0,p_date) :
                year2.append(s_year2)
                mon2.append(s_mon2) 
        else :
            p_date = 31 + int(blank_cnt)
            for y in range(0,p_date) :
                year2.append(s_year2)
                mon2.append(s_mon2) 
     
 
        # 날씨 추출하기
        #weather2=[]
        weather_2=[]
        w2= soup.select('table.table_develop > tbody > tr')
        for i in range(0,11) :
            if i % 2 != 0 :
                #print(w1[i].get_text().replace("\n",""))
                try :
                    weather_2.append(w1[i].get_text())
                except :
                    continue
        #print(weather_2)
        weather_3=[]
        for e in range(0,len(weather_2)):
            weather_3.append(weather_2[e].split("\n"))
        #print(day_4)
    
        weather_4 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, weather_3) for i in b]
        #print(day5)
        
        if s_year2 == '2018' and s_mon2 == '10' :
            for b in range(0,7) :
                weather2.append(" ")
        elif s_year2 == '2019' and s_mon2 == '01' :
            for b in range(0,7) :
                weather2.append(" ")
        elif s_year2 == '2019' and s_mon2 == '04' :
            for b in range(0,7) :
                weather2.append(" ")
        elif s_year2 == '2019' and s_mon2 == '07' :
            for b in range(0,7) :
                weather2.append(" ")
            
        for f in range(0,len(weather_4)) :
            if len(weather_4[f]) > 0:
                weather2.append(weather_4[f])
                #print(weather_4[f])
        
        

s_year: 2018
e_year: 2019
s_mon: 08
e_mon: 07
01
30


2018년 08월의 날씨 수집 중 ===================================
http://www.weather.go.kr/weather/climate/past_cal.jsp?stn=108&yy=2018&mm=08&obs=9&x=23&y=5
blank 갯수: 4


2018년 09월의 날씨 수집 중 ===================================
http://www.weather.go.kr/weather/climate/past_cal.jsp?stn=108&yy=2018&mm=09&obs=9&x=23&y=5
blank 갯수: 12


2018년 10월의 날씨 수집 중 ===================================
http://www.weather.go.kr/weather/climate/past_cal.jsp?stn=108&yy=2018&mm=10&obs=9&x=23&y=5
blank 갯수: 4


2018년 11월의 날씨 수집 중 ===================================
http://www.weather.go.kr/weather/climate/past_cal.jsp?stn=108&yy=2018&mm=11&obs=9&x=23&y=5
blank 갯수: 5


2018년 12월의 날씨 수집 중 ===================================
http://www.weather.go.kr/weather/climate/past_cal.jsp?stn=108&yy=2018&mm=12&obs=9&x=23&y=5
blank 갯수: 11


2019년 01월의 날씨 수집 중 ===================================
http://www.weather.go.kr/weather/climate/past_cal.jsp?stn=108&yy=2019&mm=01&obs=9&x=23&y=

In [3]:
#Step 8. 출력 결과를 파일에 저장하기

e_time = time.time( )
t_time = e_time - s_time

# 결과 데이터프레임화
weather_df = pd.DataFrame()
weather_df['년도']=year2
weather_df['월']=pd.Series(mon2)
weather_df['일']=pd.Series(day2)  
weather_df['요일']=pd.Series(day1)
weather_df['날씨']=pd.Series(weather2)  


# csv파일로 저장
#weather_df.to_csv(fc_name, encoding="utf-8-sig", index=False)
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'
#엑셀 파일로 저장
weather_df.to_excel(fx_name)

print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("txt 파일 저장 완료: 파일명 : %s " %fc_name)

#driver.close( )

총 소요시간은 246.7 초 입니다 
txt 파일 저장 완료: 파일명 : c:\data\2019-07-30-16-16-48-기상청날씨정보\2019-07-30-16-16-48-기상청날씨정보.csv 
